https://github.com/veribilimiokulu/udemy-apache-spark/blob/master/kurs_hazirlik/pyspark/MachineLearning/Preprocessing/PipelineOps.ipynb

## HATA AYIKLAMA İÇİN 

In [ ]:
# .setHandleInvalid("skip") KULLANILIR

In [1]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext


sc = SparkContext('local')
spark = SparkSession(sc)
sc


<SparkContext master=local appName=pyspark-shell>

In [2]:
#simpledata veri seti üzerinde çalıştık
df=spark.read.csv('simpledata.csv',header=True , inferSchema = True, sep = ",")
df.toPandas().head(2)

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200


In [3]:
#kütüphaneleri yükleyelim
from pyspark.ml.feature import StringIndexer,  VectorAssembler, StandardScaler,OneHotEncoder

In [4]:
#fonskiyon kütüpahanesi yükleyelim
from pyspark.sql import functions as F

In [5]:
#pipeline ve ML fonskiyonlarını yükleyelim
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel

## VERİ SETİNE ETİKET EKLEME

In [6]:
#yeni bir daha nitelik daha ekleyip, geliri 7000 tl'nin altı ve üstü olmak üzere iki yeni sınıf oluşturacağız.
#bu sınfların adları iyi ve kötü olacak şekilde 2'ye ayırmış olacağız
df1 = df.withColumn("ekonomik_durum",
    F.when(F.col("aylik_gelir") > 7000, "iyi").otherwise("kötü")
)

## VERİ SETİNİ TRAIN-TEST AYIRMA

In [7]:
train_df, test_df = df1.randomSplit([0.8, 0.2], seed=142)


## Pipeline için gerekli nesneleri oluşturma

In [9]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index") \
.setHandleInvalid("skip")

In [10]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index") \
.setHandleInvalid("skip")

In [12]:
encoder = OneHotEncoder() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

In [14]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [15]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [16]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [17]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

## Pipeline Oluşturma

In [18]:
pipeline_nesnesi = Pipeline() \
.setStages([meslek_indexer,sehir_indexer,encoder,assembler,label_indexer,scaler,lr_object])

In [19]:
pipeline_modeli = pipeline_nesnesi.fit(train_df)

In [20]:
pipeline_modeli.transform(test_df).select("label","prediction").toPandas().head()

,label,prediction
0,0.0,0.0
1,1.0,1.0
2,1.0,0.0
